In [1]:
import psycopg2
import pandas as pd
import requests 
from sqlalchemy import create_engine
from ast import literal_eval


In [34]:
#establish postgres connection
db_user = 'postgres'
db_password = ''
db_host = 'localhost'
db_port = 5432
database = 'met_data'

conn_str = f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{database}'
conn = psycopg2.connect(conn_str)
conn.autocommit = True

In [ ]:
#read in the MetObjects csv obtained from the Met's github
#contains all of the information the met has on each piece, but does not have urls for images
df = pd.read_csv('##filepath##/openaccess/MetObjects.csv')

In [ ]:
#turn MetObjects into postgres table
engine = create_engine(conn_str)
df.to_sql('raw_object_info', engine)

In [21]:
#look at first five items in order to get column names and quick look at data
query = """SELECT *
            FROM raw_object_info
            LIMIT 5;"""
first_5 = pd.read_sql(query, con = conn)

In [23]:
#column names have spaces that are best removed
fixed_cols = [str(col.replace(' ', '_').lower()).replace("'", '"') for col in first_5.columns]

In [25]:
#goes through the columns in postgres and fixes them 
for col in range(len(first_5.columns)):
    command = f"""ALTER TABLE raw_object_info
                RENAME COLUMN "{first_5.columns[col]}" TO {fixed_cols[col]};"""
    cursor = conn.cursor()
    try:
        cursor.execute(command)
    except:
        print('already done')
    cursor.close()
    

index
index
object_number
object_number
is_highlight
is_highlight
is_timeline_work
is_timeline_work
is_public_domain
is_public_domain
object_id
object_id
gallery_number
gallery_number
department
department
accessionyear
accessionyear
object_name
object_name
title
title
culture
culture
period
period
dynasty
dynasty
reign
reign
portfolio
portfolio
constiuent_id
constiuent_id
artist_role
artist_role
artist_prefix
artist_prefix
artist_display_name
artist_display_name
artist_display_bio
artist_display_bio
artist_suffix
artist_suffix
artist_alpha_sort
artist_alpha_sort
artist_nationality
artist_nationality
artist_begin_date
artist_begin_date
artist_end_date
artist_end_date
artist_gender
artist_gender
artist_ulan_url
artist_ulan_url
artist_wikidata_url
artist_wikidata_url
object_date
object_date
object_begin_date
object_begin_date
object_end_date
object_end_date
medium
medium
dimensions
dimensions
credit_line
credit_line
geography_type
geography_type
city
city
state
state
county
county
countr

In [27]:
#want to store artist information in separate postgres table
#create list of columns with artist in name
artist_columns = [col  for col in fixed_cols if 'artist' in col]

['artist_role',
 'artist_prefix',
 'artist_display_name',
 'artist_display_bio',
 'artist_suffix',
 'artist_alpha_sort',
 'artist_nationality',
 'artist_begin_date',
 'artist_end_date',
 'artist_gender',
 'artist_ulan_url',
 'artist_wikidata_url']

In [33]:
#for each artist column, I needed to expand the strings separated with pipes of  artists associated with piece
#for each column, iterated through using list comprehension and split on pipe if value wasnt null 
#not the most efficient way to do this, but route I took
artist_display_bio = [[i for i in df.iloc[row,20].split('|')] if df['artist_display_bio'].isna()[row] == False  else None for row in range(len(df))]
 

In [34]:
artist_suffix = [[i for i in df.iloc[row,21].split('|')] if df['artist_suffix'].isna()[row] == False  else None for row in range(len(df))]


In [35]:
artist_alpha_sort =[[i for i in df.iloc[row,22].split('|')] if df['artist_alpha_sort'].isna()[row] == False  else None for row in range(len(df))]


In [36]:
artist_nationality = [[i for i in df.iloc[row,23].split('|')] if df['artist_nationality'].isna()[row] == False  else None for row in range(len(df))]


In [37]:
artist_begin_date = [[i for i in df.iloc[row,24].split('|')] if df['artist_begin_date'].isna()[row] == False  else None for row in range(len(df))]


In [38]:
artist_end_date = [[i for i in df.iloc[row,25].split('|')] if df['artist_end_date'].isna()[row] == False  else None for row in range(len(df))]


In [39]:
artist_gender = [[i for i in df.iloc[row,26].split('|')] if df['artist_gender'].isna()[row] == False  else None for row in range(len(df))]


In [40]:
artist_ulan_url = [[i for i in df.iloc[row,27].split('|')] if df['artist_ulan_url'].isna()[row] == False  else None for row in range(len(df))]


In [41]:
artist_wikidata_url = [[i for i in df.iloc[row,28].split('|')] if df['artist_wikidata_url'].isna()[row] == False  else None for row in range(len(df))]


In [42]:
prefix = [[i for i in df.iloc[row,18].split('|')] if df['artist_prefix'].isna()[row] == False  else None for row in range(len(df))]


In [43]:
artist_display_names =  [[i for i in df.iloc[row,19].split('|')] if df['artist_display_name'].isna()[row] == False  else None for row in range(len(df))]


In [44]:
artist_roles =  [[i for i in df.iloc[row,17].split('|')] if df['artist_role'].isna()[row] == False  else None for row in range(len(df))]



In [45]:
#replace old artist columns with new list ones
df['artist_role'] = artist_roles
df['artist_prefix'] = prefix
df['artist_display_name'] = artist_display_names
df['artist_display_bio'] = artist_display_bio
df['artist_suffix'] =artist_suffix
df['artist_alpha_sort'] = artist_alpha_sort
df['artist_nationality'] = artist_nationality
df['artist_begin_date'] = artist_begin_date
df['artist_end_date'] = artist_end_date
df['artist_gender'] = artist_gender
df['artist_ulan_url'] = artist_ulan_url
df['artist_wikidata_url'] = artist_wikidata_url

In [ ]:
#function to split tags pipe lists into real lists, returns None if there is an error
def replace(x):
    try:
        return [i for i in x.split('|')]
    except:
        return None


In [ ]:
#use function to expand tags columns
df['tags'] = df.tags.map(replace)
df['tags_wikidata_url'] = df.tags_wikidata_url.map(replace)
df['tags_aat_url'] = df.tags_aat_url.map(replace)

In [ ]:
#turn MetObjects into postgres table
engine = create_engine(conn_str)
df.to_sql('object_information', engine)